In [49]:
import os
import copy
import math

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from ydata_profiling import ProfileReport
import plotly.express as px

# Problem Exploration

Doctors have the complicated task of identifying disease(s) a patient may have from information they can collect. This information can come in the form of:

- Reported symptoms from patients (from a clinical interview).
- Clinical history.
- Conducting a physical exam.
- Conducting diagnostic tests:
  - Biopsy.
  - Colonscopy.
  - CT scan.
  - Electrocardiogram (ECG).
  - ...
- Consulting with other clinicians.

## Problem as a ML Optimisation Objective

**Primary Goal:** Given a One Hot Encoded vector (of symptoms) find the most likely disease they have.

Future Goal: Given a One Hot Encoded vector (of symptoms), produce a set of diseases (could be a set of size 1) that the patient is likely to have. This may require additional data (especially for evaluation).

## What does Success Look Like?

Today's CAD (Computer Aided Diagnosis) systems have been shown to achieve up to a 90% hit rate (sensitivity = TP / (TP + FN)). I.e. they get the correct prognosis of a patient with a disease up to 90% of the time.

Models that target a single disease tend to perform better and can achieve in the high 90s for most metrics (accuracy, sensitivity, recall, F1-score etc).

Models that have been used include:

- KNN
- ANN
- Decision Trees and Random Forests
- Genetic Algorithms
- Naive Bayes

Given the more limited dataset and compute power, if I can achieve sensitivity >= 80% I will consider that a success.

References:

- [Computer-aided diagnosis Wikipedia](<https://en.wikipedia.org/wiki/Computer-aided_diagnosis#:~:text=Today's%20CAD%20systems%20cannot%20detect,a%20False%20Positive%20(FP)>)
- [Computer-aided diagnosis systems: a comparative study of classical machine learning versus deep learning-based approaches](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10205571/)

## Assumptions

- The patient has a disease in our set of diseases.
- All patients have a disease. There is no "no disease" prognosis.
- The patient is only experiencing symptoms in our set of symptoms.
- Making a diagnosis accurately only requires knowing if symptom(s) exist (binary exists or not) and not a metric of its extent/degree.


In [2]:
train_df = pd.read_csv("./dataset/original/Training.csv")
train_df.drop("Unnamed: 133", inplace=True, axis=1)

for col in train_df.columns[:-1]:
    train_df[col] = train_df[col].astype("bool")

train_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,True,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Fungal infection
1,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Fungal infection
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Fungal infection
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Fungal infection
4,True,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Fungal infection


In [3]:
len(train_df)

4920

In [4]:
test_df = pd.read_csv("./dataset/original/Testing.csv")

for col in test_df.columns[:-1]:
    test_df[col] = test_df[col].astype("bool")

test_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,True,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Fungal infection
1,False,False,False,True,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Allergy
2,False,False,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,GERD
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,Chronic cholestasis
4,True,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,Drug Reaction


In [5]:
len(test_df)

42

In [6]:
print("Number of unique diseases:", train_df["prognosis"].nunique())
print("Number of unique diseases:", test_df["prognosis"].nunique())
print("Non common prognosis:", set(train_df["prognosis"]) ^ set(test_df["prognosis"]))

Number of unique diseases: 41
Number of unique diseases: 41
Non common prognosis: set()


In [7]:
# count each prognosis
prognosis_count = train_df["prognosis"].value_counts()
prognosis_count

prognosis
Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
All

In [8]:
print(
    "All prognosises are equally distributed:", len(set(prognosis_count.to_list())) == 1
)

All prognosises are equally distributed: True


In [9]:
print(
    "Number of instances to move to test set per disease:",
    math.ceil(((4920 - 42) * 0.2) / 41),
)

Number of instances to move to test set per disease: 24


In [10]:
indexes_to_remove = []
sample_dfs = []

for prognosis in list(train_df["prognosis"].unique()):
    sample_df = train_df[train_df["prognosis"] == prognosis].sample(24, random_state=42)
    sample_dfs.append(sample_df)

    indexes_to_remove.extend(sample_df.index.to_list())

In [11]:
test_df = pd.concat([test_df] + sample_dfs)

In [12]:
train_df.drop(indexes_to_remove, inplace=True)

In [13]:
train_df.to_csv("./dataset/clean/train.csv", index=False)

In [14]:
test_df.to_csv("./dataset/clean/test.csv", index=False)

In [15]:
if not os.path.exists("symptoms-prognosis-datase-report.html"):
    profile = ProfileReport(
        train_df, title="Symptoms Prognosis Dataset Report", explorative=True
    )
    profile.to_file("symptoms-prognosis-datase-report.html")

## Questions

### 1. What proportion of the training set is duplicates?

In [107]:
duplicates_df = train_df.groupby(train_df.columns.tolist(), as_index=False).size()
print(
    "Percentage of duplicates in train set:",
    round(((sum(duplicates_df["size"]) - len(duplicates_df)) / len(train_df)) * 100, 2),
)

Percentage of duplicates in train set: 92.28


/var/folders/4l/8tynsjk14vg7p9s4qx1t_pb00000gn/T/ipykernel_17935/1958499503.py:1: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/4l/8tynsjk14vg7p9s4qx1t_pb00000gn/T/ipykernel_17935/1958499503.py:1: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/4l/8tynsjk14vg7p9s4qx1t_pb00000gn/T/ipykernel_17935/1958499503.py:1: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 2. What is the mean duplicate overlap between different prognoses?

The following code shows that there is no duplicate rows across different prognoses. Where a duplicate is the exact same **set** of symptoms across two rows.

The duplicates only exist within a given prognosis.

In [68]:
class TwoKeyDict:
    def __init__(self) -> None:
        self.dictionary = {}

    def get_value(self, key1, key2):
        if (key1, key2) in self.dictionary:
            return self.dictionary[(key1, key2)]

        if (key2, key1) in self.dictionary:
            return self.dictionary[(key2, key1)]

        return None

    def add_key(self, key1, key2, value):
        if (key2, key1) in self.dictionary:
            self.dictionary[(key2, key1)] = value

        self.dictionary[(key1, key2)] = value

In [69]:
def bool_to_int(bools):
    return int("".join(str(int(i)) for i in reversed(bools)), 2)

In [70]:
def is_same_symptoms(symptoms1: list[bool], symptoms2: list[bool]):
    return bool_to_int(symptoms1) == bool_to_int(symptoms2)

In [96]:
train_df["prognosis"].nunique(), len(train_df[train_df["prognosis"] == "Allergy"]), len(
    train_df
)

(41, 96, 3936)

In [121]:
def incorrect_calc():
    k = 96
    n = 3963  # this val was incorrect too

    t = 1

    for i in range(1, 40):
        t *= n - ((i + 1) * k)

    t *= k

    return t


def get_num_total_iterations():
    k = len(train_df[train_df["prognosis"] == "Allergy"])
    n = len(train_df)

    t = 0

    for i in range(1, train_df["prognosis"].nunique()):
        t += n - ((i) * k)

    t *= k

    return t


print(len(str(get_num_total_iterations())), get_num_total_iterations())
print(len(str(incorrect_calc())), incorrect_calc())

7 7557120
127 1245745207643194525483614406318008952385787316699404501117610092091579276975732493665396827796013887603435238781839290312500000


In [116]:
print("Actual number of iterations (given by tqdm):", 7557120)

Actual number of iterations (given by tqdm): 7557120


In [82]:
symptom_pair_to_num_overlaps = TwoKeyDict()
proccessed_prognosis = set()

with tqdm(total=get_num_total_iterations()) as pbar:
    for prognosis in tqdm(train_df["prognosis"].unique()):
        prognosis_df = train_df[train_df["prognosis"] == prognosis]

        for index1, row1 in prognosis_df.iterrows():
            for index2, row2 in train_df[
                (train_df["prognosis"] != prognosis)
                & (~train_df["prognosis"].isin(proccessed_prognosis))
            ].iterrows():
                prognosis2 = row2["prognosis"]

                if symptom_pair_to_num_overlaps.get_value(prognosis, prognosis2) is None:
                    symptom_pair_to_num_overlaps.add_key(prognosis, prognosis2, 0)

                if is_same_symptoms(row1[:-1].tolist(), row2[:-1].tolist()):
                    symptom_pair_to_num_overlaps.add_key(
                        prognosis,
                        prognosis2,
                        symptom_pair_to_num_overlaps.get_value(prognosis, prognosis2) + 1,
                    )
                    
                pbar.update(1)
                    
        proccessed_prognosis.add(prognosis)

  0%|          | 0/1245745207643194525483614406318008952385787316699404501117610092091579276975732493665396827…

  0%|          | 0/41 [00:00<?, ?it/s]

In [118]:
set(symptom_pair_to_num_overlaps.dictionary.values())

{0}

# Model Building

## Feature Engineering

- Using a LLM to categorise symptoms into:
  - Diagnostic test required for symptoms (self-reporting, diagnostic test: ECG, CT Scan etc)
- Correlation of one symptom with another to try to find unindentified symptoms
  - This would be useful to know to predict the success of a Naive Bayes model


In [16]:
# verify if the columns are in the same order in the train and test dataset
for i, col in enumerate(train_df.columns):
    if col != test_df.columns[i]:
        print(col, test_df.columns[i])

In [18]:
var_corr_df = None

if not os.path.exists("var_corr_df.csv"):
    var_to_var_corr = TwoKeyDict()
    cols = train_df.columns[:-1]

    for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
        for j, col in enumerate(cols):
            for k in range(j + 1, len(cols)):
                if var_to_var_corr.get_value(col, cols[k]) is None:
                    var_to_var_corr.add_key(col, cols[k], int(row[col] == row[cols[k]]))
                else:
                    var_to_var_corr.add_key(
                        col,
                        cols[k],
                        var_to_var_corr.get_value(col, cols[k])
                        + int(row[col] == row[cols[k]]),
                    )

    var_to_var_corr_val = copy.deepcopy(var_to_var_corr.dictionary)

    for key in var_to_var_corr_val:
        var_to_var_corr_val[key] = var_to_var_corr_val[key] / len(train_df)

    var_corr_df = pd.DataFrame(
        [[x[0][0], x[0][1], x[1]] for x in var_to_var_corr_val.items()],
        columns=["var1", "var2", "corr"],
    )
    var_corr_df.to_csv("var_corr_df.csv", index=False)
else:
    var_corr_df = pd.read_csv("var_corr_df.csv")

  0%|          | 0/3936 [00:00<?, ?it/s]

## Explanation of Correlation Distribution

This makes sense as the y-data report shows that the majority of instances per disease are recorded as false (~85%).

Hence, there would be a high likelihood of two symptoms not being present in the same instance, just because they aren't likely to be present in the first place.

It would be interesting to investigate:

### 1. Which symptoms have the lowest correlation? 
Given the previous explanation this may indicate having one symptom is counter to having another.


In [123]:
# get the 10 lowest values in var_corr_df
var_corr_df.sort_values("corr").head(10)

,var1,var2,corr
140,skin_rash,vomiting,0.500508
1395,vomiting,lethargy,0.522104
1388,vomiting,fatigue,0.532012
143,skin_rash,fatigue,0.536585
1813,fatigue,loss_of_balance,0.538618
1438,vomiting,dizziness,0.547510
1810,fatigue,swelling_joints,0.561992
1791,fatigue,neck_pain,0.562500
1809,fatigue,stiff_neck,0.562500
1849,fatigue,painful_walking,0.562500


In [47]:
fig = px.histogram(
    var_corr_df["corr"],
    x="corr",
    nbins=25,
)
fig.show()

### 2. What is the correlation between two symptoms, only for positive samples?

In [128]:
def get_pos_var_corr_df():
    pos_var_corr_df = None

    if not os.path.exists("pos_var_corr_df.csv"):
        var_to_var_corr = TwoKeyDict()
        cols = train_df.columns[:-1]

        for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
            for j, col in enumerate(cols):
                for k in range(j + 1, len(cols)):
                    if var_to_var_corr.get_value(col, cols[k]) is None:
                        val = 0 if row[col] == 0 else int(row[col] == row[cols[k]])

                        var_to_var_corr.add_key(col, cols[k], val)
                    else:
                        if row[col] == 0:
                            continue

                        var_to_var_corr.add_key(
                            col,
                            cols[k],
                            var_to_var_corr.get_value(col, cols[k])
                            + int(row[col] == row[cols[k]]),
                        )

        var_to_var_corr_val = copy.deepcopy(var_to_var_corr.dictionary)

        for key in var_to_var_corr_val:
            var_to_var_corr_val[key] = var_to_var_corr_val[key] / len(train_df)

        pos_var_corr_df = pd.DataFrame(
            [[x[0][0], x[0][1], x[1]] for x in var_to_var_corr_val.items()],
            columns=["var1", "var2", "corr"],
        )
        pos_var_corr_df.to_csv("pos_var_corr_df.csv", index=False)
    else:
        pos_var_corr_df = pd.read_csv("pos_var_corr_df.csv")
        
    return pos_var_corr_df

In [129]:
pos_var_corr_df = get_pos_var_corr_df()

  0%|          | 0/3936 [00:00<?, ?it/s]

In [132]:
fig = px.histogram(
    pos_var_corr_df["corr"],
    x="corr",
    nbins=50,
)
fig.show()

In [136]:
pos_var_corr_df.sort_values("corr", ascending=False).head(10)

,var1,var2,corr
1753,fatigue,high_fever,0.199949
1408,vomiting,nausea,0.197917
1413,vomiting,abdominal_pain,0.174797
3997,loss_of_appetite,yellowing_of_eyes,0.160315
1763,fatigue,loss_of_appetite,0.156504
1409,vomiting,loss_of_appetite,0.155234
3702,yellowish_skin,abdominal_pain,0.154726
1388,vomiting,fatigue,0.153963
3893,nausea,loss_of_appetite,0.136179
1776,fatigue,malaise,0.135163
